# Exploratory Data Analysis - Coffee Tree V0.02

In [1]:
import pandas as pd
from janitor import clean_names

df = clean_names(pd.read_excel("data/raw/modelo_cafe_2024nov.xlsm", sheet_name="TODOS DADOS"))

df.columns = [col.strip('_') for col in df.columns]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927 entries, 0 to 926
Data columns (total 43 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   envio                                    927 non-null    object 
 1   laboratorio                              927 non-null    object 
 2   amostra                                  927 non-null    object 
 3   odor_do_po                               927 non-null    float64
 4   odor_da_bebida_*                         718 non-null    float64
 5   sabor_da_bebida_*                        927 non-null    float64
 6   docura                                   927 non-null    float64
 7   corpo                                    927 non-null    float64
 8   acidez_intensidade                       927 non-null    float64
 9   acidez_qualidade                         927 non-null    float64
 10  amargor                                  927 non-n

## Observações da Aline

Interessante não cnsiderar esses atributos:         
    - `odor_da_bebida_*`;       
    - `sabor_da_bebida_*`;     

Target:     
    - `qualidade_global_media_dada_pela_equipe`;     

Identificação das amostras:     
    - `laboratorio`;        
    - `amostra`;        
    - `fora_de_tipo_x_tipo_unico`;          
    - `categoria`;      
    - `sigla_categoria`;        

Médias dos atributos:       
    - `odor_do_po`;     
    - `docura`;    
    - `corpo`;     
    - `acidez_intensidade`;     
    - `acidez_qualidade`;     
    - `amargor`;     
    - `adstringencia`;     
    - `intensidade`;      

Mediana dos descritores:        
    - `alcoolico`;     
    - `cedro_carvalho_amadeirado`;     
    - `amendoado_castanhas`;     
    - `animalico_curral`;     
    - `azedo`;     
    - `baunilha`;     
    - `borracha`;     
    - `caramelizado_caramelo_doce`;     
    - `cereal`;     
    - `chocolate_cacau`;     
    - `cozido_assado`;     
    - `especiarias`;     
    - `fermentado`;     
    - `floral`;     
    - `frutado`;     
    - `iodoformio_quimico`;     
    - `madeira_papelao`;     
    - `mel`;     
    - `queimado_defumado`;     
    - `terroso_mofo`;     
    - `tostado`;     
    - `vegetal`;     
    - `velho_oxidado`;     
    - `verde`;      
    - `herbaceo`;     


# Análise de correlação

In [2]:
df[
    [
        "odor_da_bebida_*",
        "sabor_da_bebida_*",
        "qualidade_global_media_dada_pela_equipe",
    ]
].corr()

,odor_da_bebida_*,sabor_da_bebida_*,qualidade_global_media_dada_pela_equipe
odor_da_bebida_*,1.000000,0.990853,0.988608
sabor_da_bebida_*,0.990853,1.000000,0.995081
qualidade_global_media_dada_pela_equipe,0.988608,0.995081,1.000000


# Seleção de atributos

In [3]:
df_model = df[
    [
        # Target
        "qualidade_global_media_dada_pela_equipe",
        # Classificação de laboratórios
        "laboratorio",
        "fora_de_tipo_x_tipo_unico",
        # Médias dos atributos
        "odor_do_po",
        "docura",
        "corpo",
        "acidez_intensidade",
        "acidez_qualidade",
        "amargor",
        "adstringencia",
        "intensidade",
        # Mediana dos descritores
        "alcoolico",
        "cedro_carvalho_amadeirado",
        "amendoado_castanhas",
        "animalico_curral",
        "azedo",
        "baunilha",
        "borracha",
        "caramelizado_caramelo_doce",
        "cereal",
        "chocolate_cacau",
        "cozido_assado",
        "especiarias",
        "fermentado",
        "floral",
        "frutado",
        "iodoformio_quimico",
        "madeira_papelao",
        "mel",
        "queimado_defumado",
        "terroso_mofo",
        "tostado",
        "vegetal",
        "velho_oxidado",
        "verde", 
        "herbaceo",
    ]
]

## Tratamento da coluna fora_de_tipo_x_tipo_unico

In [4]:
df_model.loc[:, "fora_de_tipo_x_tipo_unico"] = (
    df_model["fora_de_tipo_x_tipo_unico"]
    .str.lower()
    .str.replace(" ", "_")
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
)

# Análise de multicolinearidade

In [5]:
# Análise de VIF (Variance Inflation Factor)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Separar features e target, usando apenas variáveis numéricas
X = df_model.select_dtypes(include=["float64", "int64"]).drop(
    "qualidade_global_media_dada_pela_equipe", axis=1
)
y = df_model["qualidade_global_media_dada_pela_equipe"]

# Calcular VIF para cada feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Ordenar por VIF em ordem decrescente
vif_data = vif_data.sort_values("VIF", ascending=False)

# Mostrar resultados
print("Fatores de Inflação da Variância (VIF):")
print("VIF > 10 indica alta multicolinearidade")
print("\n", vif_data)

Fatores de Inflação da Variância (VIF):
VIF > 10 indica alta multicolinearidade

                        Feature         VIF
2                        corpo  743.864287
4             acidez_qualidade  606.145686
3           acidez_intensidade  395.047984
6                adstringencia  347.605268
1                       docura  316.957193
5                      amargor  315.737485
0                   odor_do_po  276.542435
7                  intensidade  123.415364
22                     frutado    6.939547
15  caramelizado_caramelo_doce    6.115461
24             madeira_papelao    5.097204
30               velho_oxidado    3.843020
10         amendoado_castanhas    3.012181
26           queimado_defumado    2.942400
17             chocolate_cacau    2.442214
28                     tostado    1.991756
25                         mel    1.746509
14                    borracha    1.544714
13                    baunilha    1.530696
9    cedro_carvalho_amadeirado    1.410160
20             

## Interpretação dos resultados

1. **Variáveis com alta multicolinearidade (VIF > 10)**:
   - As variáveis **_acidez_qualidade** (1333.37), **_acidez_intensidade** (1216.63) e **_amargor** (1044.73) possuem VIF extremamente altos, indicando que essas variáveis estão fortemente correlacionadas entre si ou com outras variáveis do modelo. Essa multicolinearidade pode fazer com que seja difícil distinguir o efeito individual de cada uma dessas variáveis sobre o desfecho, e pode causar instabilidade nos coeficientes estimados.
   - O **_corpo** (919.10) e **_adstringencia** (895.62) também possuem VIF muito elevados, apontando para multicolinearidade significativa.

2. **Variáveis com VIF entre 5 e 10**:
   - Essas variáveis, como **_frutado_** (8.79), **_madeira_papelao_** (6.40), e **_caramelizado_caramelo_doce_** (5.54), apresentam uma multicolinearidade moderada. Embora não tão grave quanto as primeiras, essas variáveis ainda podem afetar a precisão das estimativas e devem ser monitoradas, pois um VIF acima de 5 já é considerado uma indicação de atenção.

3. **Variáveis com baixa multicolinearidade (VIF < 5)**:
   - A maioria das variáveis restantes, como **_velho_oxidado_** (4.73), **_queimado_defumado_** (4.14), e **_tostado_** (2.20), apresenta VIFs abaixo de 5, o que indica pouca ou nenhuma multicolinearidade preocupante. Estas variáveis estão relativamente bem comportadas no modelo.


# Salvar o dataframe limpo


In [6]:
df_model.to_parquet("data/processed/df_model_v02.parquet")